# **QnA Dataset Generation for EcoSortify Chatbot Model**

<center><img src="https://img.freepik.com/vector-gratis/contaminacion-basura-basura-objetos-basura-aislados_1308-34354.jpg"/></center>

### **Install Vertex AI SDK for Python and other dependencies**

In [ ]:
!pip install --upgrade --user -q google-cloud-aiplatform

### **Authenticate notebook environment (Colab only)**

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using Vertex AI Workbench.

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### **Set Google Cloud project information and initialize Vertex AI SDK**

To get started using Vertex AI, you must have an existing Google Cloud project and enable the Vertex AI API.

In [ ]:
import os

from google import genai
from google.genai import types

PROJECT_ID = "ecosortify-459004"
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### **Import Libraries**

By importing these, you gain access to the necessary tools for generating data and interacting with the base model.

In [ ]:
from collections import Counter
import json
import random
import httpx

# Vertex AI SDK
from google.cloud import aiplatform
from google.cloud.aiplatform.metadata import context
from google.cloud.aiplatform.metadata import utils as metadata_utils
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
for model in client.models.list():
    print(model)

name='publishers/google/models/chat-bison' display_name=None description=None version='002' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=None output_token_limit=None supported_actions=None default_checkpoint_id=None checkpoints=None
name='publishers/google/models/text-bison' display_name=None description=None version='002' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=None output_token_limit=None supported_actions=None default_checkpoint_id=None checkpoints=None
name='publishers/google/models/imageclassification-efficientnet' display_name=None description=None version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=None output_token_limit=None supported_actions=None default_checkpoint_id=None checkpoints=None
name='publishers/google/

### **Setup Prompt Structure and Source Documents**

Defines AI instructions, source documents, and number of Q&A pairs for dataset generation.

In [ ]:
system_instruction="""Kamu adalah AI yang mengkhususkan diri dalam menghasilkan dataset pelatihan chatbot tanya jawab berkualitas tinggi untuk fine tuning large language model berbahasa indonesia. Tujuan Kamu adalah membantu saya dalam membuat data sintetis untuk aplikasi edukasi pengetahuan dan pemilahan sampah.
Gunakan input prompt dari saya sebagai konteks untuk membuat dataset tanya jawabnya. Buatkan pertanyaan yang berbobot dan edukatif. Untuk bagian jawaban oleh model, gunakan bahasa yang khas yang digunakan bot untuk melayani pertanyaan pengguna. Bagian jawaban oleh model harus jelas dan padat tetapi tetap komprehensif. Gunakan bahasa indonesia yang tidak terlalu kaku atau formal.
Jika saya menyuruh kamu membuat dataset sintesis tersebut, buatkan dalam struktur json seperti dibawah ini tetapi dalam format 1 file json line, dan buatkan langsung kode json line saja tanpa ada teks tambahan diluar, agar saya bisa menyimpannya langsung di file .jsonl.

{
   "contents":[
      # all Q&A content is here
      {
         "role":"user",
         "parts":[
            {
               "text":"Question text..." # This is question content
            }
         ]
      },
      {
         "role":"model",
            {
               "text":"Answer text..." # This is answer content
            }
         ]
      }
      ... # next QnA content here
   ]
}
"""

In [ ]:
# Source document dataset
knlh_doc_url = "https://storage.googleapis.com/ecosortify-assets/raw-document/KNLH-P3S_removed.pdf"
guide1_doc_url = "https://storage.googleapis.com/ecosortify-assets/raw-document/Materi%20Guide%20%2B%20LM%20Dataset%20%201.pdf"
guide2_doc_url = "https://storage.googleapis.com/ecosortify-assets/raw-document/Materi%20Guide%20%2B%20LM%20Dataset%202.pdf"

# Retrieve and encode the PDF byte
knlh_doc_data = httpx.get(knlh_doc_url).content
guide1_doc_data = httpx.get(guide1_doc_url).content
guide2_doc_data = httpx.get(guide2_doc_url).content

# Determine the number of qna pair counts
qna_pairs_count = 150
prompt=f"Buatkan {qna_pairs_count} tanya jawab berdasarkan dokumen tersebut."

### **Tuning Model Paramaters**

Configures the parameters for the Gemini model, such as the temperature and the maximum number of output tokens, which influence the style and length of the generated text.

In [ ]:
# https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig

base_model = "publishers/google/models/gemini-2.5-pro-preview-05-06"

generation_config = types.GenerateContentConfig(
    temperature=0.5,
    top_p=0.95,
    system_instruction=system_instruction,
    max_output_tokens=65535,
)

### **Generate Dataset (Run The Prompt)**



Runs the configured prompt using the Gemini model, incorporating the PDF document data and the specified prompt text to generate the Q&A dataset based on the provided context and instructions.

In [ ]:
# https://googleapis.github.io/python-genai/genai.html#genai.models.Models.generate_content

response_knlh = client.models.generate_content(
    model=base_model,
    config=generation_config,
    contents=[
        types.Part.from_bytes(
            data=knlh_doc_data,
            mime_type='application/pdf',
        ),
        prompt
    ],
)

print(response_knlh.text)

```json
{"contents":[ {"role":"user", "parts":[{"text":"Mengapa pengelolaan sampah di kota-kota besar seringkali masih menjadi masalah yang sulit dikendalikan?"}]}, {"role":"model", "parts":[{"text":"Tentu! Pengelolaan sampah di kota besar jadi rumit karena beberapa faktor nih. Pertama, jumlah penduduk dan kegiatan ekonomi yang terus meningkat bikin volume sampah juga makin banyak. Kedua, timbulan sampah yang nggak terkendali ini bisa jadi konsekuensi dari aktivitas manusia dan industri. Nah, ini kemudian berdampak ke lingkungan perkotaan, seperti masalah keindahan, kesehatan masyarakat, bahkan bisa memicu bencana kayak ledakan gas metan atau tanah longsor."}]}]}
{"contents":[ {"role":"user", "parts":[{"text":"Apa fokus utama pengelolaan sampah yang umumnya dilakukan oleh dinas terkait saat ini?"}]}, {"role":"model", "parts":[{"text":"Jadi begini, pengelolaan sampah yang diselenggarakan oleh dinas terkait itu kebanyakan masih fokus pada pengumpulan dan pengangkutan sampah langsung ke T

In [ ]:
# https://googleapis.github.io/python-genai/genai.html#genai.models.Models.generate_content

response_guide1 = client.models.generate_content(
    model=base_model,
    config=generation_config,
    contents=[
        types.Part.from_bytes(
            data=guide1_doc_data,
            mime_type='application/pdf',
        ),
        prompt
    ],
)

print(response_guide1.text)

```json
{"contents":[{"role":"user","parts":[{"text":"Halo EcoSortify, bisa jelaskan apa itu sampah menurut panduan ini?"}]},{"role":"model","parts":[{"text":"Tentu! Menurut panduan EcoSortify, sampah adalah material sisa atau buangan dari aktivitas manusia maupun proses alam, yang sudah nggak punya nilai pakai atau fungsi lagi buat pemiliknya. Jadi, intinya sih, material yang udah kehilangan nilainya dalam sistem konsumsi kita."}]}]}
{"contents":[{"role":"user","parts":[{"text":"Mengapa pola konsumsi dan produksi manusia disebut mempercepat terbentuknya sampah?"}]},{"role":"model","parts":[{"text":"Oke, jadi begini, semakin banyak kita mengonsumsi dan memproduksi barang, semakin banyak juga bahan baku yang diambil dari alam, diolah, dan akhirnya digunakan. Nah, ujung-ujungnya, semakin besar juga kemungkinan barang-barang itu jadi limbah. Proses ini kayak siklus yang terus meningkat, makanya pembentukan sampah jadi lebih cepat."}]}]}
{"contents":[{"role":"user","parts":[{"text":"Apa si

In [ ]:
# https://googleapis.github.io/python-genai/genai.html#genai.models.Models.generate_content

response_guide2 = client.models.generate_content(
    model=base_model,
    config=generation_config,
    contents=[
        types.Part.from_bytes(
            data=guide2_doc_data,
            mime_type='application/pdf',
        ),
        prompt
    ],
)

print(response_guide2.text)

```json
{"contents":[{"role":"user","parts":[{"text":"Apakah benar sampah organik selalu aman untuk lingkungan?"}]},{"role":"model","parts":[{"text":"Hmm, anggapan itu kurang tepat nih. Sampah organik memang lebih mudah terurai, tapi kalau dibuang sembarangan dan nggak dikelola dengan baik, malah bisa menghasilkan gas metana yang berbahaya buat lingkungan, lho. Tumpukannya di TPA juga bisa mencemari air tanah dan bikin bau nggak sedap."}]}]}
{"contents":[{"role":"user","parts":[{"text":"Katanya bank sampah itu cuma buat orang yang kekurangan, bener nggak sih?"}]},{"role":"model","parts":[{"text":"Eits, jangan salah! Bank sampah itu bukan cuma soal ekonomi, tapi lebih ke gerakan peduli lingkungan. Siapapun, dari latar belakang apa aja, bisa ikutan kok. Selain nabung sampah, di sana juga bisa jadi tempat belajar dan kumpul komunitas buat gaya hidup yang lebih ramah lingkungan."}]}]}
{"contents":[{"role":"user","parts":[{"text":"Percuma dong milah sampah daur ulang, kan ujung-ujungnya dib

### **Dataset Pre-Processing**

Preprocesses the generated Q&A dataset by extracting the "contents" from each of the three model responses, combining them into a single list, and structuring the combined data into a dictionary with a single "contents" key.

In [ ]:
def extract_contents(response):
    # Remove the wrapper ```json ... ```
    cleaned = response.text.replace("```json", "").replace("```", "")

    # Split by newline (each line is assumed to be a JSON object)
    json_objects = cleaned.strip().split("\n")

    # Combine all contents from each JSON object into one list
    combined_contents = []
    for obj in json_objects:
        parsed = json.loads(obj)
        combined_contents.extend(parsed["contents"])

    return combined_contents

# Process all three responses
all_contents = []
for response in [response_knlh, response_guide1, response_guide2]:
    all_contents.extend(extract_contents(response))

# Wrap everything in a new JSON object
final_json = {"contents": all_contents}

print(final_json)

{'contents': [{'role': 'user', 'parts': [{'text': 'Mengapa pengelolaan sampah di kota-kota besar seringkali masih menjadi masalah yang sulit dikendalikan?'}]}, {'role': 'model', 'parts': [{'text': 'Tentu! Pengelolaan sampah di kota besar jadi rumit karena beberapa faktor nih. Pertama, jumlah penduduk dan kegiatan ekonomi yang terus meningkat bikin volume sampah juga makin banyak. Kedua, timbulan sampah yang nggak terkendali ini bisa jadi konsekuensi dari aktivitas manusia dan industri. Nah, ini kemudian berdampak ke lingkungan perkotaan, seperti masalah keindahan, kesehatan masyarakat, bahkan bisa memicu bencana kayak ledakan gas metan atau tanah longsor.'}]}, {'role': 'user', 'parts': [{'text': 'Apa fokus utama pengelolaan sampah yang umumnya dilakukan oleh dinas terkait saat ini?'}]}, {'role': 'model', 'parts': [{'text': 'Jadi begini, pengelolaan sampah yang diselenggarakan oleh dinas terkait itu kebanyakan masih fokus pada pengumpulan dan pengangkutan sampah langsung ke Tempat Pemro

### **Save to JSON Line Format (.jsonl)**

In [ ]:
# Simpan ke file
with open("all_qna_pairs.jsonl", "w", encoding="utf-8") as f:
    f.write(json.dumps(final_json))